In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem(word):
    return stemmer.stem(word.lower())

In [ ]:
import numpy as np

def bag_of_words(tokenize_sentence, all_words):
    tokenize_sentence = [stem(w) for w in tokenize_sentence]
    
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenize_sentence:
            bag[idx] = 1.0
            
    return bag

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving intends.json to intends.json


In [ ]:
import json

with open('intends.json', 'r') as file:
    intends = json.load(file)

print(intends)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [ ]:
all_words = []
tags = []
xy = []

for intent in intends['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w= tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))


In [ ]:
ignore_words = ['?',',','!','.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [ ]:
import numpy as np

x_train = []
y_train = []
for (pattern_sentence,tag) in xy:
    bag = bag_of_words(pattern_sentence,all_words)
    x_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label) #cross_entropy loss
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [ ]:
batch_size = 8

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
class NeuralNet(nn.Module):

    def __init__(self,input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out - self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        #no activation and no softmax
        return out

In [ ]:
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
print(input_size, len(all_words))
print(output_size, tags)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size)

54 54
7 ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [ ]:
learning_rate = 0.001
num_ephocs = 1000

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_ephocs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    if (epoch+1)%100 == 0:
        print(f'epoch [{epoch+1}/{num_ephocs}], loss = {loss.item():.4f}')

print(f'final loss = {loss.item():.4f}')


In [ ]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hidden_size":hidden_size,
    "all_words":all_words,
    "tags":tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f"training complete. file saved to {FILE}")

training complete. file saved to data.pth


In [ ]:
import random

In [ ]:
FILE = "data.pth"
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "sam"
print("Lets chat! type'quit' to exit")
while True:
    sentence = input('You: ')
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    x = bag_of_words(sentence, all_words)
    x = x.reshape(1,x.shape[0])
    x = torch.from_numpy(x).to(device)
    output = model(x)

    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() >0.75:
        for intent in intends['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not Understand ....")

Lets chat! type'quit' to exit
You: hello
sam: Hi there, what can I do for you?
You: goodbye
sam: Have a nice day
You: give me a glass of water
sam: Why did the hipster burn his mouth? He drank the coffee before it was cool.
You: item
sam: I do not Understand ....
You: items
sam: I do not Understand ....
You: items
sam: I do not Understand ....
You: 'Which items do you have?
sam: We sell coffee and tea
You: what items do you have?
sam: We have coffee and tea
You: quit
